In [1]:
%cd ..

c:\Users\admin\Crawl_buffett


In [2]:
from src.dividend import *

In [5]:
dividendClass = GetDividend(path_save='tests/')
id_company = 9888

In [6]:
dividendClass.getDividendTableFromPdf(id_company, '2015', 'Q4')

,2015,Q1,Q2,Q3,Q4
0,2015,—,3.00,一,7.00


In [11]:

for file in os.listdir('tests/' + f"Data"):
    id_company = int(file)
    dividendClass.getDividend(id_company= id_company,
                     return_df=False, 
                     save_file= True)

1301 --------
1332 --------


page has no images - all vector content will be rasterized at 400 DPI, losing some resolution and likely increasing file size. Use --oversample to adjust the DPI.
[tesseract] lots of diacritics - possibly poor OCR
[tesseract] lots of diacritics - possibly poor OCR
[tesseract] lots of diacritics - possibly poor OCR
[tesseract] lots of diacritics - possibly poor OCR
The output file size is 3.30× larger than the input file.
Possible reasons for this include:
--force-ocr was issued, causing transcoding.
Optimization was disabled.



1333 --------
1375 --------
1376 --------
9636 --------
9672 --------
9831 --------
9873 --------
9888 --------
9994 --------
9995 --------
9996 --------
9997 --------


In [18]:
pd.read_csv("tests/Data/9997/docs/dividend.csv")

,Year,Q1,Q2,Q3,Q4,time_split_Q1,time_split_Q2,time_split_Q3,time_split_Q4
0,2022,―,10.00,―,10.00,NaN,['2_12_2022'],NaN,['29_6_2023']
1,2021,―,9.50,―,9.50,NaN,['3_12_2021'],NaN,['29_6_2022']
2,2020,―,8.00,―,8.50,NaN,['4_12_2020'],NaN,['28_6_2021']
3,2019,―,8.00,―,8.00,NaN,['4_12_2019'],NaN,['26_6_2020']
4,2018,―,7.50,―,7.50,NaN,['4_12_2018'],NaN,['27_6_2019']
5,2017,―,6.25,―,6.25,NaN,['4_12_2017'],NaN,['28_6_2018']
6,2016,―,6.25,―,6.25,NaN,['2_12_2016'],NaN,['29_6_2017']
7,2015,―,6.25,―,6.25,NaN,['4_12_2015'],NaN,['29_6_2016']
8,2014,―,6.25,―,6.25,NaN,['5_12_2014'],NaN,['29_6_2015']
9,2013,―,12.50,―,6.25,NaN,['6_12_2013'],NaN,['30_6_2014']


In [12]:
pd.read_csv(f"docs/DividendShares.csv")

,Symbol,DividendShares
0,1301,False
1,1332,False
2,1333,False
3,1375,False
4,1376,False
...,...,...
1722,9994,Done
1723,9995,Done
1724,9996,Done
1725,9997,Done
